In [1]:
!pip install autogluon.tabular==1.1.1 -q  

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
aiobotocore 2.13.2 requires botocore<1.34.132,>=1.34.70, but you have botocore 1.29.165 which is incompatible.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.8.2 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatib

In [2]:
# !pip uninstall lightgbm -y -q
# !pip install lightgbm --install -option=--gpu -q

In [3]:
!pip install rapidfuzz -q

In [4]:

import os
import sys
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

from autogluon.tabular import TabularPredictor
# from autogluon.features.generators import AutoMLPipelineFeatureGenerator

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from rapidfuzz import process, fuzz

from textblob import TextBlob

import re

import spacy
from sklearn.feature_extraction.text import CountVectorizer


import lightgbm as lgb
import catboost as cb
import xgboost as xgb

from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

import json


from math import sqrt

from lightgbm import early_stopping, log_evaluation


pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option("display.max_rows", None)

In [5]:
train_df = pd.read_csv('/kaggle/input/datathon-2024/train.csv')
test_df = pd.read_csv('/kaggle/input/datathon-2024/test_x.csv')
il_ilce = pd.read_csv('/kaggle/input/trkiye-il-ile-listesi/il_ilce.csv')

stopwords = '/kaggle/input/tr-trke-stopwords-turkish/turkce_stopwords.json'

train_df.head()

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,Universite Kacinci Sinif,Universite Not Ortalamasi,Daha Once Baska Bir Universiteden Mezun Olmus,Lise Adi,Lise Adi Diger,Lise Sehir,Lise Turu,Lise Bolumu,Lise Bolum Diger,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,Burs Aldigi Baska Kurum,Baska Kurumdan Aldigi Burs Miktari,Anne Egitim Durumu,Anne Calisma Durumu,Anne Sektor,Baba Egitim Durumu,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,Bilgisayar Muhendisligi,3,3.50-3,Hayır,Ankara Ataturk Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.50-3,Hayır,NaN,NaN,Üniversite,Hayır,0,Üniversite,Emekli,0,1,Evet,Bilkent IEEE Bilgisayar Toplulugu 2013-2014 Ba...,Evet,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,Elektrik Elektronik Mühendisliği,3,3.00-2.50,Hayır,Betül Can Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.00 - 4.00,Hayır,NaN,NaN,Üniversite,Evet,Kamu,Yüksek Lisans / Doktora,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,Finans Yönetimi ve Pazarlama,1,NaN,Hayır,Batem,NaN,Ankara,Meslek lisesi,Elektirk,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,İlkokul Mezunu,Evet,Özel Sektör,6,Evet,Tema,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,Bilgisayar Mühendisliği,3,3.00-2.50,Hayır,Cumhuriyet Fen Lisesi,NaN,Diyarbakır,Fen lisesi,Sayısal,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,10,Evet,Güzel Konuşma ve yazma kulübü,Hayır,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,NaN,3.00-2.50,Hayır,Samanyolu Lisesi,NaN,Ankara,Özel lisesi,TM,NaN,3.00 - 4.00,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4


In [6]:
with open(stopwords, 'r', encoding='utf-8') as file:
    data = json.load(file)

stop_words_list = data.get('stopwords', [])

print(stop_words_list)
print(len(stop_words_list))

['a', 'acaba', 'acep', 'adamakıllı', 'adeta', 'ait', 'altmýþ', 'altmış', 'altı', 'ama', 'amma', 'anca', 'ancak', 'arada', 'artýk', 'aslında', 'aynen', 'ayrıca', 'az', 'açıkça', 'açıkçası', 'bana', 'bari', 'bazen', 'bazý', 'bazı', 'başkası', 'baţka', 'belki', 'ben', 'benden', 'beni', 'benim', 'beri', 'beriki', 'beş', 'beş', 'beţ', 'bilcümle', 'bile', 'bin', 'binaen', 'binaenaleyh', 'bir', 'biraz', 'birazdan', 'birbiri', 'birden', 'birdenbire', 'biri', 'birice', 'birileri', 'birisi', 'birkaç', 'birkaçı', 'birkez', 'birlikte', 'birçok', 'birçoğu', 'birþey', 'birþeyi', 'birşey', 'birşeyi', 'birţey', 'bitevi', 'biteviye', 'bittabi', 'biz', 'bizatihi', 'bizce', 'bizcileyin', 'bizden', 'bize', 'bizi', 'bizim', 'bizimki', 'bizzat', 'boşuna', 'bu', 'buna', 'bunda', 'bundan', 'bunlar', 'bunları', 'bunların', 'bunu', 'bunun', 'buracıkta', 'burada', 'buradan', 'burası', 'böyle', 'böylece', 'böylecene', 'böylelikle', 'böylemesine', 'böylesine', 'büsbütün', 'bütün', 'cuk', 'cümlesi', 'da', 'daha', '

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 44 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               65125 non-null  int64  
 1   Degerlendirme Puani                                        65124 non-null  float64
 2   Cinsiyet                                                   64956 non-null  object 
 3   Dogum Tarihi                                               64948 non-null  object 
 4   Dogum Yeri                                                 64334 non-null  object 
 5   Ikametgah Sehri                                            63088 non-null  object 
 6   Universite Adi                                             64993 non-null  object 
 7   Universite Turu                                            64870 non-null  object 
 8   Burslu

In [8]:
train_df.isnull().sum()

Basvuru Yili                                                     0
Degerlendirme Puani                                              1
Cinsiyet                                                       169
Dogum Tarihi                                                   177
Dogum Yeri                                                     791
Ikametgah Sehri                                               2037
Universite Adi                                                 132
Universite Turu                                                255
Burslu ise Burs Yuzdesi                                      42685
Burs Aliyor mu?                                                  0
Bölüm                                                          231
Universite Kacinci Sinif                                       374
Universite Not Ortalamasi                                     2753
Daha Once Baska Bir Universiteden Mezun Olmus                37345
Lise Adi                                                      

In [9]:
class Config:

    catboost_params = {
            'random_state': 42,
            'loss_function': 'RMSE',
            'eval_metric': 'RMSE',
            'learning_rate': 0.2,
            'iterations': 1000,
            'task_type': 'GPU',
    }


    target_col = 'Degerlendirme Puani'

    drop_cols = ['Burslu ise Burs Yuzdesi','Daha Once Baska Bir Universiteden Mezun Olmus', 'Lise Adi Diger',
                    'Lise Bolum Diger', 'Uye Oldugunuz Kulubun Ismi', 'Stk Projesine Katildiniz Mi?', 'Ingilizce Seviyeniz?',
                    'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']


sehir_cols = ['Dogum Yeri', 'Ikametgah Sehri', 'Lise Sehir']

In [10]:


il_ilce_dict = il_ilce.set_index('ilce')['il'].to_dict()

# Function to map Dogum Yeri using the dictionary
def map_il_using_dict(dogum_yeri):
    for ilce in il_ilce_dict:
        if ilce in str(dogum_yeri):
            return il_ilce_dict[ilce]
    return dogum_yeri

def safe_standardize_university_name(name):
    if isinstance(name, str):
        name = name.upper()
        name = re.sub(r'Ü', 'U', name)
        name = re.sub(r'İ', 'I', name)
        name = re.sub(r'Ö', 'O', name)
        name = re.sub(r'Ç', 'C', name)
        name = re.sub(r'Ş', 'S', name)
        name = re.sub(r'Ğ', 'G', name)
        name = re.sub(r'[^A-Z ]', '', name)
    return name

def safe_standardize_name(name):
    if isinstance(name, str):
        name = name.upper()
        name = re.sub(r'Ü', 'U', name)
        name = re.sub(r'İ', 'I', name)
        name = re.sub(r'Ö', 'O', name)
        name = re.sub(r'Ç', 'C', name)
        name = re.sub(r'Ş', 'S', name)
        name = re.sub(r'Ğ', 'G', name)
        name = re.sub(r'[^A-Z ]', '', name)
    return name

def group_similar_departments(name):
    if isinstance(name, str):

        name = re.sub(r'\s+', '', name.upper())

        if 'TIP' in name:
            return 'TIP FAKULTESI'
        elif 'ISLETME' in name:
            return 'ISLETME'
        elif 'HUKUK' in name:
            return 'HUKUK'
        elif 'FIZIK' in name:
            return 'FIZIK'
        elif 'BILGISAYARMUHENDISLIGI' in name:
            return 'BILGISAYAR MUHENDISLIGI'
        elif 'ENDUSTRIMUHENDISLIGI' in name:
            return 'ENDUSTRI MUHENDISLIGI'
        elif 'MAKINEMUHENDISLIGI' in name:
            return 'MAKINE MUHENDISLIGI'
        elif 'MAKINAMUHENDISLIGI' in name:
            return 'MAKINE MUHENDISLIGI'
        elif 'GIDAMUHENDISLIGI' in name:
            return 'GIDA MUHENDISLIGI'
        elif 'MEKATRONIKMUHENDISLIGI' in name:
            return 'MEKATRONIK MUHENDISLIGI'
        elif 'BIYOMEDIKALMUHENDISLIGI' in name:
            return 'BIYOMEDIKAL MUHENDISLIGI'
        elif 'ELEKTRIKELEKTRONIKMUHENDISLIGI' in name:
            return 'ELEKTRIK ELEKTRONIK MUHENDISLIGI'
        elif 'ELEKTRIKMUHENDISLIGI' in name:
            return 'ELEKTRIK ELEKTRONIK MUHENDISLIGI'
        elif 'ELEKTRONIKMUHENDISLIGI' in name:
            return 'ELEKTRIK ELEKTRONIK MUHENDISLIGI'
        elif 'ELEKTRONIKHABERLESMEMUHENDISLIGI' in name:
            return 'ELEKTRIK ELEKTRONIK MUHENDISLIGI'
        elif 'ELEKTRONIKVEHABERLESMEMUHENDISLIGI' in name:
            return 'ELEKTRIK ELEKTRONIK MUHENDISLIGI'
        elif 'INSAATMUHENDISLIGI' in name:
            return 'INSAAT MUHENDISLIGI'
        elif 'ICMIMARLIK' in name:
            return 'IC MIMARLIK'
        elif 'ULUSLARARASIILISKILER' in name:
            return 'ULUSLARARASI ILISKILER'
        elif 'ENERJIMUHENDISLIGI' in name:
            return 'ENERJI MUHENDISLIGI'
        elif 'KIMYAMUHENDISLIGI' in name:
            return 'KIMYA MUHENDISLIGI'
        elif 'IKTISAT' in name:
            return 'IKTISAT'
        elif 'HARITAMUHENDISLIGI' in name:
            return 'HARITA MUHENDISLIGI'
        elif 'PSIKOLOJİ' in name:
            return 'PSIKOLOJİ'
        elif 'HEMSIRELIK'  in name:
            return 'HEMSIRELIK'

        elif 'MADEN' in name:
            return 'MADEN MUHENDISLIGI'
        elif 'SOSYOLO' in name:
            return 'SOSYOLOJI'
        elif 'MALIYE' in name:
            return 'MALIYE'
        elif 'ILAH' in name:
            return 'ILAHIYAT'
        elif 'ECZAC' in name:
            return 'ECZACILIK'
        elif 'DISH' in name:
            return 'DIS HEKIMLIGI'
        elif 'YONETIMBILIS' in name:
            return 'YONETIM BILISIM SISTEMLERI'
        elif 'MATEMATIKMU' in name:
            return 'MATEMATIK MUHENDISLIGI'
        elif 'COGRAFYA' in name:
            return 'COGRAFYA'
        elif 'GAZETE' in name:
            return 'GAZETECILIK'
        elif 'FELSEFE' in name:
            return 'FELSEFE'
        elif 'EDEBIYAT' in name:
            return 'EDEBIYAT'

        elif 'METALURJI' in name:
            return 'METALURJI MALZEME MUHENDISLIGI'
        elif 'MALZEME' in name and 'METALURJI' not in name:
            return 'MALZEME MUHENDISLIGI'
        elif 'SINIFOGRET' in name:
            return 'SINIF OGRETMENLIGI'
        elif 'CALISMAEKO' in name:
            return 'CALISMA EKONOMISI'
        elif 'HEMSI' in name:
            return 'HEMSIRELIK'
        elif 'KAMUY' in name:
            return 'KAMU YONETIMI'
        elif 'JEOLO' in name:
            return 'JEOLOJI MUHENDISLIGI'
        elif 'BESLENME' in name:
            return 'BESLENME VE DIYETETIK'

        elif 'EKONOME' in name:
            return 'EKONOMETRI'
        elif 'CEVREMU' in name:
            return 'CEVRE MUHENDISLIGI'
        elif 'YAZILIMMU' in name:
            return 'YAZILIM MUHENDISLIGI'
        elif 'ENERJISIS' in name:
            return 'ENERJI SISTEMLERI MUHENDISLIGI'
        elif 'MEKATRONIKMU' in name:
            return 'MEKATRONIK MUHENDISLIGI'
        elif 'REHBERLIKVE' in name:
            return 'REHBERLIK VE PSIKOLOJIK DANISMANLIK'

        elif 'OKULON' in name:
            return 'OKUL ONCESI OGRETMENLIGI'
        elif 'MOLEKU' in name:
            return 'MOLEKULER BIYOLOJI VE GENETIK'

        elif 'HALKLA' in name:
            return 'HALKLA ILISKILER'
    return name

def group_similar_burs(name):
    if isinstance(name, str):

        name = re.sub(r'\s+', '', name.upper())
        name = name.strip()

        if 'KREDI' in name:
            return 'KYK'
        elif 'DEVLET' in name:
            return 'KYK'
        elif 'KREDİ' in name:
            return 'KYK'

    return name

def group_similar_lise(lise):
    if isinstance(lise, str):
        # Clean and prepare the input string
        lise = lise.strip().lower()
        lise = re.sub(r'\s+', '', lise)

        # Define categories and their keywords
        categories = {
            'sayısal': ['sayı', 'sayi', 'fizik', 'fen', 'fm', 'mf'],
            'eşit ağırlık': ['esit', 'eşit', 'matematik', 'ea', 'esıt', 'türkçe', 'turkce',
                             'eşi̇tağirlik', 'eşi̇t-ağirlik', 'eşi̇tagirlik', 'tmeşi̇tağirlik',
                             'meslek-eşi̇tağirlik', 'esi̇tagirlik', 'esi̇tagrlik', 'eşi̇tağirlikbölümü',
                             'eşi̇tağirliktm', 'eşi̇tağrlik', 'eş,itağırlık'],
            'sözel': ['sözel', 'sosyal', 'ts', 'sozel', 'söz', 'soz'],
            'dil bölümü': ['dil', 'ingilizce', 'yabancı', 'yabanci', 'di̇lalani'],
            'endüstri meslek lisesi': ['elektrik', 'elektronik', 'bilişim', 'bilgisayar', 'makine',
                                       'makina', 'yazilim', 'yazılım', 'veri', 'veritabani', 'endu',
                                       'endü', 'otom', 'bi̇li̇şi̇mteknoloji̇leri̇', 'bilisimteknolojileri',
                                       'inşaat', 'insaat', 'muhasebe', 'yiyecek', 'icecek', 'içecek', 'turi', 'yi̇yeceki̇çecekhi̇zmetleri̇','konaklama'],
            'imam hatip lisesi': ['imamhatip', 'imam hatip lisesi', 'imam'],
            'öğretmen': ['öğretmen', 'ogretmen', 'ögretmen'],
            'sosyal bilimler': ['sosyaşbilimler', 'soyalbilimler'],
            'çocuk gelişimi' : ['çocukgelişimi', 'cocuk', 'gelisim', 'gelişim', 'cocukgelişimi', 'cocukgelisimi'],
            'askeri': ['askeri'],
        }

        # Match the input string with defined categories
        for category, keywords in categories.items():
            if any(keyword in lise for keyword in keywords):
                return category

def lda_score(train_df):

    turkish_stop_words = [
        "ve", "bir", "bu", "için", "ile", "de", "da", "ama", "veya", "ise", "çünkü", "ancak",
        "her", "ne", "gibi", "ki", "ya", "olarak", "çok", "olan", "daha", "değil", "kadar",
        "bile", "sonra", "birçok", "bazı", "herkes", "biri", "birisi", "bazıları", "bu yüzden"
    ]

    # Selecting non-null responses from the column
    text_data = train_df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].dropna()

    # Re-initialize TF-IDF Vectorizer with manually defined Turkish stop words
    vectorizer = TfidfVectorizer(stop_words=turkish_stop_words, max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(text_data)

    # Apply Latent Dirichlet Allocation (LDA) to discover topics within the responses
    lda = LatentDirichletAllocation(
        n_components=5,
        random_state=0,
        )
    lda.fit(tfidf_matrix)

    # Extracting the top words from each topic
    n_top_words = 10
    feature_names = vectorizer.get_feature_names_out()
    topics = {}
    for topic_idx, topic in enumerate(lda.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        topics[f"Topic {topic_idx+1}"] = top_features


    topic_scores = lda.transform(tfidf_matrix)
    non_null_indices = train_df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].dropna().index

    # Creating a DataFrame from the topic scores and aligning with the original indices
    topic_df = pd.DataFrame(lda.transform(tfidf_matrix), index=non_null_indices, columns=[f"Topic_{i+1}_Score" for i in range(lda.n_components)])

    train_df = train_df.join(topic_df, how='left')

    return train_df


def girisimcilik_process(train_df):

    experience_column = train_df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].fillna('')

    train_df['character_count'] = experience_column.apply(len)
    train_df['word_count'] = experience_column.apply(lambda x: len(x.split()))
    train_df['average_word_length'] = train_df['character_count'] / train_df['word_count'].replace(0, 1)

    train_df['sentiment'] = experience_column.apply(lambda x: TextBlob(x).sentiment.polarity)

    return train_df

def stk_process(train_df):

    experience_column = train_df["Hangi STK'nin Uyesisiniz?"].fillna('')

    train_df['character_count'] = experience_column.apply(len)
    train_df['word_count'] = experience_column.apply(lambda x: len(x.split()))
    train_df['average_word_length'] = train_df['character_count'] / train_df['word_count'].replace(0, 1)

    train_df['sentiment'] = experience_column.apply(lambda x: TextBlob(x).sentiment.polarity)

    return train_df


train_df['Bölüm'] = train_df['Bölüm'].str.lower().str.strip()
test_df['Bölüm'] = test_df['Bölüm'].str.lower().str.strip()

reference_list = train_df['Bölüm'].value_counts().index.tolist()
reference_list_test = test_df['Bölüm'].value_counts().index.tolist()


def standardize_bölüm(value, reference_list):
    result = process.extractOne(value, reference_list, scorer=fuzz.partial_ratio)
    if result is not None:
        match, score, _ = result
        if score > 85:
            return match
    return value

train_df['Bölüm'] = train_df['Bölüm'].apply(lambda x: standardize_bölüm(x, reference_list))
test_df['Bölüm'] = test_df['Bölüm'].apply(lambda x: standardize_bölüm(x, reference_list_test))



In [11]:
def process_data(df):

    global sehir_cols

    train_df = df.copy()

    train_df = train_df.drop(columns=['id'], errors='ignore')
    # train_df = train_df.drop(columns=['Dogum Tarihi'], errors='ignore')

    train_df['Dogum Tarihi'] = train_df['Dogum Tarihi'].replace(['1/1/70 2:00'], pd.NaT)
    train_df['Dogum Tarihi'] = pd.to_datetime(train_df['Dogum Tarihi'], errors='coerce')
    train_df['Dogum Tarihi'] = train_df['Dogum Tarihi'].dt.strftime('%Y-%m-%d')

    train_df['Basvuru Tarihi_2'] = pd.to_datetime(train_df['Basvuru Yili'].astype(str) + '-01-01', errors='coerce')
    train_df['Age'] = (train_df['Basvuru Tarihi_2'] - pd.to_datetime(train_df['Dogum Tarihi'], errors='coerce')).dt.days // 365

    train_df = train_df.drop(columns=['Basvuru Tarihi_2'], errors='ignore')

    for col in sehir_cols:
        train_df[col] = train_df[col].str.strip().str.title()
        train_df[col] = train_df[col].str.split('/').str[0].str.strip()
        train_df[col] = train_df[col].str.split(' ').str[0].str.strip()
        train_df[col] = train_df[col].str.split('-').str[0].str.strip()
        train_df[col] = train_df[col].str.split('_').str[0].str.strip()
        train_df[col] = train_df[col].str.split(',').str[0].str.strip()
        train_df[col] = train_df[col].replace('İstanbulbakırköy', 'İstanbul')
        train_df[col] = train_df[col].replace('Istanbul', 'İstanbul')
        train_df[col] = train_df[col].replace('Aliağaizmir', 'İzmir')
        train_df[col] = train_df[col].replace('İzmiR', 'İzmir')
        train_df[col] = train_df[col].replace('İzmirkonak', 'İzmir')
        train_df[col] = train_df[col].replace('Yenimahalleankara', 'Ankara')
        train_df[col] = train_df[col].replace('Bursayıldırım', 'Bursa')
        train_df[col] = train_df[col].replace('Yumurtalıkadana', 'Adana')
        train_df[col] = train_df[col].replace('Karaisalıadana', 'Adana')
        train_df[col] = train_df[col].replace('Adanaseyhan', 'Adana')
        train_df[col] = train_df[col].replace('Konyaselçuklu', 'Konya')
        train_df[col] = train_df[col].replace('Diyarbakir', 'Diyarbakır')
        train_df[col] = train_df[col].replace('Diyarbakırdicle', 'Diyarbakır')
        train_df[col] = train_df[col].replace('Si̇Lvan ', 'Silvan')
        train_df[col] = train_df[col].replace('Sanliurfa', 'Şanlıurfa')
        train_df[col] = train_df[col].replace('Şanliurfa', 'Şanlıurfa')
        train_df[col] = train_df[col].replace('Ş.Urfa', 'Şanlıurfa')
        train_df[col] = train_df[col].replace('Malatyamerkez', 'Malatya')
        train_df[col] = train_df[col].replace('Kahramanmaras', 'Kahramanmaraş')
        train_df[col] = train_df[col].replace('K.Maras', 'Kahramanmaraş')
        train_df[col] = train_df[col].replace('K.Maraş', 'Kahramanmaraş')
        train_df[col] = train_df[col].replace('Agri', 'Ağrı')
        train_df[col] = train_df[col].replace('Ağri', 'Kahramanmaraş')
        train_df[col] = train_df[col].replace('Balikesir', 'Balıkesir')
        train_df[col] = train_df[col].replace('BalikesiR', 'Balıkesir')
        train_df[col] = train_df[col].replace('Orduünye', 'Ordu')
        train_df[col] = train_df[col].replace('Eskisehir', 'Eskişehir')
        train_df[col] = train_df[col].replace('Adiyaman', 'Adıyaman')
        train_df[col] = train_df[col].replace('Aydin', 'Aydın')
        train_df[col] = train_df[col].replace('Bartin', 'Bartın')
        train_df[col] = train_df[col].replace('Elazig', 'Elazığ')
        train_df[col] = train_df[col].replace('Tekirdag', 'Tekirdağ')
        train_df[col] = train_df[col].replace('Sirnak', 'Şırnak')
        train_df[col] = train_df[col].replace('Şirnak', 'Şırnak')
        train_df[col] = train_df[col].replace('Nigde', 'Niğde')
        train_df[col] = train_df[col].replace('Kirikkale', 'Kırıkkale')
        train_df[col] = train_df[col].replace('Kirklareli', 'Kırklareli')
        train_df[col] = train_df[col].replace('Igdir', 'Iğdır')
        train_df[col] = train_df[col].replace('İzmiR', 'İzmir')
        train_df[col] = train_df[col].replace('Eflani', 'Karabük')
        train_df[col] = train_df[col].replace('Gazipaşa', 'Antalya')
        train_df[col] = train_df[col].replace('Gaziemir', 'İzmir')
        train_df[col] = train_df[col].replace('Gaziantepnizip', 'Gaziantep')
        train_df[col] = train_df[col].replace('Alabama', 'Yurtdışı')
        train_df[col] = train_df[col].replace('G.O.Paşa', 'İstanbul')
        train_df[col] = train_df[col].replace('Nizip', 'Gaziantep')
        train_df[col] = train_df[col].replace('Ni̇ksar', 'Tokat')
        train_df[col] = train_df[col].replace('Ni̇lüfer', 'Bursa')
        train_df[col] = train_df[col].replace('Nottingham', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Naziili', 'Aydın')
        train_df[col] = train_df[col].replace('Nusaybi̇n', 'Mardin')
        train_df[col] = train_df[col].replace('Ezine', 'Çanakkale')
        train_df[col] = train_df[col].replace('Oguzeli', 'Gaziantep')
        train_df[col] = train_df[col].replace('Oltu', 'Erzurum')
        train_df[col] = train_df[col].replace('Etimesgut', 'Ankara')
        train_df[col] = train_df[col].replace('Orhangazi', 'Bursa')
        train_df[col] = train_df[col].replace('Orsu', 'Ordu')
        train_df[col] = train_df[col].replace('Ortaca', 'Muğla')
        train_df[col] = train_df[col].replace('Espi̇ye', 'Giresun')
        train_df[col] = train_df[col].replace('Osman', 'Bartın')
        train_df[col] = train_df[col].replace('Eskişeir', 'Eskişehir')
        train_df[col] = train_df[col].replace('Odunpazarı', 'Eskişehir')
        train_df[col] = train_df[col].replace('Narman', 'Erzurum')
        train_df[col] = train_df[col].replace('Gazi̇osmanpaşa', 'İstanbul')
        train_df[col] = train_df[col].replace('Gediz', 'Kütahya')
        train_df[col] = train_df[col].replace('Golcuk', 'Kocaeli')
        train_df[col] = train_df[col].replace('Melbourne', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Gokcebey', 'Zonguldak')
        train_df[col] = train_df[col].replace('Glresun', 'Giresun')
        train_df[col] = train_df[col].replace('Meriç', 'Edirne')
        train_df[col] = train_df[col].replace('Araç', 'Kastamonu')
        train_df[col] = train_df[col].replace('Aladağ', 'Adana')
        train_df[col] = train_df[col].replace('Geyve', 'Sakarya')
        train_df[col] = train_df[col].replace('Merzi̇fon', 'Amasya')
        train_df[col] = train_df[col].replace('Gerger', 'Adıyaman')
        train_df[col] = train_df[col].replace('Gercüş.', 'Batman')
        train_df[col] = train_df[col].replace('Mi̇las', 'Muğla')
        train_df[col] = train_df[col].replace('Mrsin', 'Mersin')
        train_df[col] = train_df[col].replace('Gemli̇k', 'Bursa')
        train_df[col] = train_df[col].replace('Gemerek', 'Sivas')
        train_df[col] = train_df[col].replace('Muradiye', 'Van')
        train_df[col] = train_df[col].replace('Muratbey', 'Bursa')
        train_df[col] = train_df[col].replace('Gelibolu', 'Çanakkale')
        train_df[col] = train_df[col].replace('Mustafa', 'Bursa')
        train_df[col] = train_df[col].replace('Mustafakemalpaşa', 'Bursa')
        train_df[col] = train_df[col].replace('Gedizkütahya', 'Kütahya')
        train_df[col] = train_df[col].replace('Osmancık', 'Çorum')
        train_df[col] = train_df[col].replace('Mecitözü', 'Çorum')
        train_df[col] = train_df[col].replace('Eskipazar', 'Karabük')
        train_df[col] = train_df[col].replace('Erzurun', 'Erzurum')
        train_df[col] = train_df[col].replace('Sarıgöl', 'Manisa')
        train_df[col] = train_df[col].replace('Sarıkamış', 'Kars')
        train_df[col] = train_df[col].replace('Sason', 'Batman')
        train_df[col] = train_df[col].replace('Schorndorf', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Sehitkamil', 'Gaziantep')
        train_df[col] = train_df[col].replace('Selendi', 'Manisa')
        train_df[col] = train_df[col].replace('Seli̇m', 'Kars')
        train_df[col] = train_df[col].replace('Ekinözü', 'Kahramanmaraş')
        train_df[col] = train_df[col].replace('Selçukulu', 'Konya')
        train_df[col] = train_df[col].replace('Kandıra', 'Kocaeli')
        train_df[col] = train_df[col].replace('Sarikaya', 'Yozgat')
        train_df[col] = train_df[col].replace('Edremi̇t', 'Balıkesir')
        train_df[col] = train_df[col].replace('Edi̇rne', 'Edirne')
        train_df[col] = train_df[col].replace('Arsi̇n', 'Trabzon')
        train_df[col] = train_df[col].replace('Ede', 'Giresun')
        train_df[col] = train_df[col].replace('Seyi̇tgazi̇', 'Eskişehir')
        train_df[col] = train_df[col].apply(map_il_using_dict)

        train_df[col] = train_df[col].replace('Karedeniz', 'Trabzon')
        train_df[col] = train_df[col].replace('Bartin', 'Bartın')
        train_df[col] = train_df[col].replace('Baski̇l', 'Elazığ')
        train_df[col] = train_df[col].replace('Elaziğ', 'Elazığ')
        train_df[col] = train_df[col].replace('Kktc', 'Kıbrıs')
        train_df[col] = train_df[col].replace('Ki̇li̇s', 'Bartın')
        train_df[col] = train_df[col].replace('Kiziltepe	', 'Mardin')
        train_df[col] = train_df[col].replace('Kelki̇t', 'Gümüşhane')
        train_df[col] = train_df[col].replace('Gümüshane', 'Gümüşhane')
        train_df[col] = train_df[col].replace('Gumushane	', 'Gümüşhane')
        train_df[col] = train_df[col].replace('Kdz.', 'Zonguldak')
        train_df[col] = train_df[col].replace('Kaçanik', 'Kosova')
        train_df[col] = train_df[col].replace('Kazan	', 'Rusya')
        train_df[col] = train_df[col].replace('Kayserı', 'Kayseri')
        train_df[col] = train_df[col].replace('Bayindir', 'İzmir')
        train_df[col] = train_df[col].replace('Kastamonuci̇de', 'Kastamonu')
        train_df[col] = train_df[col].replace('Bağcilar', 'İstanbul')
        train_df[col] = train_df[col].replace('Karsıyaka', 'İzmir')
        train_df[col] = train_df[col].replace('Berlin', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Karadeniz', 'Trabzon')
        train_df[col] = train_df[col].replace('Kopnya', 'Konya')
        train_df[col] = train_df[col].replace('Kandira', 'Kocaeli')
        train_df[col] = train_df[col].replace('Kusadasi', 'Aydın')
        train_df[col] = train_df[col].replace('Köln', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Aralik', 'Iğdır')
        train_df[col] = train_df[col].replace('Arpacay', 'Kars')
        train_df[col] = train_df[col].replace('Ni̇ğde', 'Ni̇ğde')
        train_df[col] = train_df[col].replace('Avcilar', 'İstanbul')
        train_df[col] = train_df[col].replace('Azerbeycan', 'Azerbaycan')
        train_df[col] = train_df[col].replace('Meli̇kgazi̇', 'Kayseri')
        train_df[col] = train_df[col].replace('Mazidagi', 'Mardin')
        train_df[col] = train_df[col].replace('Mardi̇n', 'Mardin')
        train_df[col] = train_df[col].replace('Baku', 'Azerbaycan')
        train_df[col] = train_df[col].replace('Macoo', 'İran')
        train_df[col] = train_df[col].replace('M.K.Paşa', 'Bursa')
        train_df[col] = train_df[col].replace('Levent', 'İstanbul')
        train_df[col] = train_df[col].replace('Lefkoşe', 'Kıbrıs')
        train_df[col] = train_df[col].replace('Lefkoşa', 'Kıbrıs')
        train_df[col] = train_df[col].replace('Lazkiye', 'Suriye')
        train_df[col] = train_df[col].replace('Bakü', 'Azerbaycan')
        train_df[col] = train_df[col].replace('Kırgızistan', 'Kırgızistan')
        train_df[col] = train_df[col].replace('Kırcali', 'Bulgaristan')
        train_df[col] = train_df[col].replace('Kütahyatavşanlı', 'Kütahya')
        train_df[col] = train_df[col].replace('Kâhta', 'Adıyaman')
        train_df[col] = train_df[col].replace('Beyşehi̇r', 'Konya')
        train_df[col] = train_df[col].replace('Kadıöy', 'İstanbul')
        train_df[col] = train_df[col].replace('Kadıkoy', 'İstanbul')
        train_df[col] = train_df[col].replace('Erurum', 'Erzurum')
        train_df[col] = train_df[col].replace('Erci̇ş', 'Van')
        train_df[col] = train_df[col].replace('Emnönü', 'İstanbul')
        train_df[col] = train_df[col].replace('Emi̇nönü', 'İstanbul')
        train_df[col] = train_df[col].replace('Cankırı', 'Çankırı')
        train_df[col] = train_df[col].replace('Cardak', 'Denizli')
        train_df[col] = train_df[col].replace('Duzıcı', 'Osmaniye')
        train_df[col] = train_df[col].replace('Duisburg', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Doğubeyazıt', 'Ağrı')
        train_df[col] = train_df[col].replace('Doganhisar', 'Konya')
        train_df[col] = train_df[col].replace('Cekerek', 'Yozgat')
        train_df[col] = train_df[col].replace('Celikhan', 'Adıyaman')
        train_df[col] = train_df[col].replace('Charsadda', 'Pakistan')
        train_df[col] = train_df[col].replace('Deri̇k', 'Mardin')
        train_df[col] = train_df[col].replace('Ci̇dde', 'Arabistan')
        train_df[col] = train_df[col].replace('Demokratik', 'Kongo')
        train_df[col] = train_df[col].replace('Dargeçi̇t', 'Mardin')
        train_df[col] = train_df[col].replace('D.Bayazıt', 'Ağrı')
        train_df[col] = train_df[col].replace('Kemalpasa', 'İzmir')
        train_df[col] = train_df[col].replace('Siivas', 'Sivas')
        train_df[col] = train_df[col].replace('Ist', 'İstanbul')
        train_df[col] = train_df[col].replace('Sisli', 'İstanbul')
        train_df[col] = train_df[col].replace('Ispir', 'Erzurum')
        train_df[col] = train_df[col].replace('Si̇lvan', 'Diyarbakır')
        train_df[col] = train_df[col].replace('Si̇ncan', 'Ankara')
        train_df[col] = train_df[col].replace('Iscehisar', 'Afyonkarahisar')
        train_df[col] = train_df[col].replace('Afyon', 'Afyonkarahisar')
        train_df[col] = train_df[col].replace('Afyonkarahi̇sar', 'Afyonkarahisar')
        train_df[col] = train_df[col].replace('Ilgin', 'Konya')
        train_df[col] = train_df[col].replace('State', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Stockholm', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Stuttgart', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Altintaş', 'Kütahya')
        train_df[col] = train_df[col].replace('Sultanahmet', 'İstanbul')
        train_df[col] = train_df[col].replace('Sultandagi', 'Afyonkarahisar')
        train_df[col] = train_df[col].replace('Suriye', 'Suriye')
        train_df[col] = train_df[col].replace('Suudi', 'Arabistan')
        train_df[col] = train_df[col].replace('Sììrt', 'Siirt')
        train_df[col] = train_df[col].replace('Sınop', 'Sinop')
        train_df[col] = train_df[col].replace('Tacikistan', 'Tacikistan')
        train_df[col] = train_df[col].replace('Taksim', 'İstanbul')
        train_df[col] = train_df[col].replace('Taksi̇m', 'İstanbul')
        train_df[col] = train_df[col].replace('Bi̇ga', 'Çanakkale')
        train_df[col] = train_df[col].replace('Tavsanli', 'Kütahya')
        train_df[col] = train_df[col].replace('Tavşanli', 'Kütahya')
        train_df[col] = train_df[col].replace('Idil', 'Şırnak')
        train_df[col] = train_df[col].replace('Ivrindi', 'Balıkesir')
        train_df[col] = train_df[col].replace('Seydisehir', 'Konya')
        train_df[col] = train_df[col].replace('Ci̇hanbeyli̇', 'Konya')
        train_df[col] = train_df[col].replace('Kosova', 'Kosova')
        train_df[col] = train_df[col].replace('Barti̇n', 'Bartın')
        train_df[col] = train_df[col].replace('Bi̇leci̇k', 'Bi̇leci̇k')
        train_df[col] = train_df[col].replace('Kongo', 'Kongo')
        train_df[col] = train_df[col].replace('Kiziltepe', 'Mardin')
        train_df[col] = train_df[col].replace('Kirklareli̇', 'Kırklareli')
        train_df[col] = train_df[col].replace('Bakırkoy', 'İstanbul')
        train_df[col] = train_df[col].replace('Kazan', 'Rusya')
        train_df[col] = train_df[col].replace('Bol', 'Bolu')
        train_df[col] = train_df[col].replace('Bolton', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Bratosh', 'Arnavutluk')
        train_df[col] = train_df[col].replace('Bristol', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Ni̇ğde', 'Ni̇ğde')
        train_df[col] = train_df[col].replace('Burhani̇ye', 'Balıkesir')
        train_df[col] = train_df[col].replace('Pendi̇k', 'İstanbul')
        train_df[col] = train_df[col].replace('K.Çekmece', 'İstanbul')
        train_df[col] = train_df[col].replace('Ankar7', 'Ankara')
        train_df[col] = train_df[col].replace('Izmr', 'İzmir')
        train_df[col] = train_df[col].replace('Sahinbey', 'Gaziantep')
        train_df[col] = train_df[col].replace('Altıntş', 'Kütahya')
        train_df[col] = train_df[col].replace('Altinözü', 'Hatay')
        train_df[col] = train_df[col].replace('Sansun', 'Samsun')
        train_df[col] = train_df[col].replace('Tehran', 'İran')
        train_df[col] = train_df[col].replace('Hof', 'Almanya')
        train_df[col] = train_df[col].replace('Talipli', 'Giresun')
        train_df[col] = train_df[col].replace('Tervel', 'Bulgaristan')
        train_df[col] = train_df[col].replace('Golkoy', 'Ordu')
        train_df[col] = train_df[col].replace('Çamardi', 'Niğde')
        train_df[col] = train_df[col].replace('Gleize', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Teki̇rdağçorlu', 'Teki̇rdağ')
        train_df[col] = train_df[col].replace('Çekirge', 'Bursa')
        train_df[col] = train_df[col].replace('Gross', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Wuppertal', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Viyana', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Vezi̇rkörü', 'Samsun')
        train_df[col] = train_df[col].replace('Vezi̇rköprü', 'Samsun')
        train_df[col] = train_df[col].replace('Vakfikebi̇r', 'Trabzon')
        train_df[col] = train_df[col].replace('Ali̇ağa', 'İzmir')
        train_df[col] = train_df[col].replace('Ali̇aga', 'İzmir')
        train_df[col] = train_df[col].replace('Türkoğu', 'Kahramanmaraş')
        train_df[col] = train_df[col].replace('Tunceli̇', 'Tunceli̇')
        train_df[col] = train_df[col].replace('Yeni̇ce', 'Çanakkale')
        train_df[col] = train_df[col].replace('Trablus', 'Libya')
        train_df[col] = train_df[col].replace('Cumhuriyet', 'İstanbul')
        train_df[col] = train_df[col].replace('Torbali', 'İzmir')
        train_df[col] = train_df[col].replace('Yildirim', 'Bursa')
        train_df[col] = train_df[col].replace('?Irıkkale', 'Kırıkkale')
        train_df[col] = train_df[col].replace('Göstepe', 'İzmir')
        train_df[col] = train_df[col].replace('Gölbaşi', 'Ankara')
        train_df[col] = train_df[col].replace('Gıresun', 'Giresun')
        train_df[col] = train_df[col].replace('Şahi̇nbey', 'Gaziantep')
        train_df[col] = train_df[col].replace('Şanluırfa', 'Şanlıurfa')
        train_df[col] = train_df[col].replace('Abington', 'Yurtdışı')
        train_df[col] = train_df[col].replace('34239', 'Hatay')
        train_df[col] = train_df[col].replace('Üskidar', 'İstanbul')
        train_df[col] = train_df[col].replace('Ümrani̇ye', 'İstanbul')
        train_df[col] = train_df[col].replace('Şehi̇tkami̇l', 'Gaziantep')
        train_df[col] = train_df[col].replace('Şeki̇', 'Azerbaycan')
        train_df[col] = train_df[col].replace('İlkadim', 'Samsun')
        train_df[col] = train_df[col].replace('Gümüşhaciköy', 'Amasya')
        train_df[col] = train_df[col].replace('Çi̇ftli̇k', 'Niğde')
        train_df[col] = train_df[col].replace('Çinar', 'Diyarbakır')
        train_df[col] = train_df[col].replace('Çeli̇khan', 'Adıyaman')
        train_df[col] = train_df[col].replace('Çermi̇k', 'Sivas')
        train_df[col] = train_df[col].replace('Akdagmadeni', 'Yozgat')
        train_df[col] = train_df[col].replace('Şerefli̇koçhi̇sar', 'Ankara')
        train_df[col] = train_df[col].replace('Çankırı', 'Çankırı')
        train_df[col] = train_df[col].replace('Ödemi̇ş', 'İzmir')
        train_df[col] = train_df[col].replace('Akcadag', 'Malatya')
        train_df[col] = train_df[col].replace('Reyhanli', 'Hatay')
        train_df[col] = train_df[col].replace('Erzi̇ncan', 'Erzi̇ncan')
        train_df[col] = train_df[col].replace('Karşiyaka', 'İzmir')
        train_df[col] = train_df[col].replace('Gumushane', 'Gümüşhane')
        train_df[col] = train_df[col].replace('Mekke', 'Arabistan')
        train_df[col] = train_df[col].replace('Elmali', 'Antalya')
        train_df[col] = train_df[col].replace('Kirikhan', 'Hatay')
        train_df[col] = train_df[col].replace('Kizilcahamam', 'Ankara')
        train_df[col] = train_df[col].replace('Beyoglu', 'İstanbul')
        train_df[col] = train_df[col].replace('Bahcelievler', 'İstanbul')
        train_df[col] = train_df[col].replace('Osmancik', 'Çorum')
        train_df[col] = train_df[col].replace('Eyup', 'İstanbul')
        train_df[col] = train_df[col].replace('Hakkari̇', 'Hakkari̇')
        train_df[col] = train_df[col].replace('Kahire', 'Mısır')
        train_df[col] = train_df[col].replace('Akhi̇sar', 'Manisa')
        train_df[col] = train_df[col].replace('G.Antep', 'Gaziantep')
        train_df[col] = train_df[col].replace('Alaşehi̇r', 'Manisa')
        train_df[col] = train_df[col].replace('Adakli', 'Bingöl')
        train_df[col] = train_df[col].replace('Uskudar', 'İstanbul')
        train_df[col] = train_df[col].replace('Corum', 'Çorum')
        train_df[col] = train_df[col].replace('Karabuk', 'Karabük')
        train_df[col] = train_df[col].replace('Fethi̇ye', 'Muğla')
        train_df[col] = train_df[col].replace('Sultanbeli', 'İstanbul')
        train_df[col] = train_df[col].replace('Kadi̇rli̇', 'Osmaniye')
        train_df[col] = train_df[col].replace('Kagithane', 'İstanbul')
        train_df[col] = train_df[col].replace('Si̇i̇rt', 'Si̇i̇rt')
        train_df[col] = train_df[col].replace('Seydi̇şehi̇r', 'Konya')
        train_df[col] = train_df[col].replace('Seri̇k', 'Antalya')
        train_df[col] = train_df[col].replace('Artvi̇n', 'Artvi̇n')
        train_df[col] = train_df[col].replace('Cankaya', 'Ankara')
        train_df[col] = train_df[col].replace('Keçi̇ören', 'Ankara')
        train_df[col] = train_df[col].replace('Iğdir', 'Iğdır')
        train_df[col] = train_df[col].replace('Iğdir', 'Iğdır')
        train_df[col] = train_df[col].replace('Kdz', 'Zonguldak')
        train_df[col] = train_df[col].replace('Kağıthane', 'İstanbul')
        train_df[col] = train_df[col].replace('Şi̇şli̇', 'İstanbul')
        train_df[col] = train_df[col].replace('Bandirma', 'Balıkesir')
        train_df[col] = train_df[col].replace('Yüreği̇r', 'Adana')
        train_df[col] = train_df[col].replace('Nazi̇lli̇', 'Aydın')
        train_df[col] = train_df[col].replace('Altindag', 'Ankara')
        train_df[col] = train_df[col].replace('Bi̇tli̇s', 'Bi̇tli̇s')
        train_df[col] = train_df[col].replace('Kadikoy', 'İstanbul')
        train_df[col] = train_df[col].replace('Kirşehi̇r', 'Kırşehi̇r')
        train_df[col] = train_df[col].replace('Inegöl', 'Bursa')
        train_df[col] = train_df[col].replace('Kadi̇köy', 'İstanbul')
        train_df[col] = train_df[col].replace('Adapazari', 'Sakarya')
        train_df[col] = train_df[col].replace('Kutahya', 'Kütahya')
        train_df[col] = train_df[col].replace('Eyüp', 'İstanbul')
        train_df[col] = train_df[col].replace('Polatli', 'Ankara')
        train_df[col] = train_df[col].replace('Kocasi̇nan', 'Kayseri')
        train_df[col] = train_df[col].replace('Yeni̇mahalle', 'Ankara')
        train_df[col] = train_df[col].replace('Fati̇h', 'İstanbul')
        train_df[col] = train_df[col].replace('Balikesi̇r', 'Balıkesi̇r')
        train_df[col] = train_df[col].replace('Izmit', 'Kocaeli')
        train_df[col] = train_df[col].replace('İzmi̇t', 'Kocaeli')
        train_df[col] = train_df[col].replace('Iskenderun', 'Hatay')
        train_df[col] = train_df[col].replace('Bakirkoy', 'İstanbul')
        train_df[col] = train_df[col].replace('Altindağ', 'Ankara')
        train_df[col] = train_df[col].replace('Bakirköy', 'İstanbul')
        train_df[col] = train_df[col].replace('Eminönü', 'İstanbul')
        train_df[col] = train_df[col].replace('Kadiköy', 'İstanbul')
        train_df[col] = train_df[col].replace('Kirsehir', 'Kırşehi̇r')
        train_df[col] = train_df[col].replace('Nevsehir', 'Nevşehir')
        train_df[col] = train_df[col].replace('Usak', 'Uşak')
        train_df[col] = train_df[col].replace('Izmir', 'İzmir')
        train_df[col] = train_df[col].replace('Usak', 'Uşak')
        train_df[col] = train_df[col].replace('Mugla', 'Muğla')
        train_df[col] = train_df[col].replace('Mus', 'Muş')
        train_df[col] = train_df[col].replace('Hakkâri', 'Hakkari')
        train_df[col] = train_df[col].replace('Di̇yarbakir', 'Di̇yarbakır')
        train_df[col] = train_df[col].replace('Girne', 'Kıbrıs')
        train_df[col] = train_df[col].replace('Kocamustafapaşa', 'İstanbul')
        train_df[col] = train_df[col].replace('İZmir', 'İzmir')
        train_df[col] = train_df[col].replace('Feriköy', 'İstanbul')
        train_df[col] = train_df[col].replace('Florya', 'İstanbul')
        train_df[col] = train_df[col].replace('Gazimağusa', 'Kıbrıs')
        train_df[col] = train_df[col].replace('Lefke', 'Kıbrıs')
        train_df[col] = train_df[col].replace('Odtü', 'Ankara')
        train_df[col] = train_df[col].replace('Oslo', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Eryaman', 'Ankara')
        train_df[col] = train_df[col].replace('Gazi̇mağusa', 'Kıbrıs')
        train_df[col] = train_df[col].replace('Nürnberg', 'Yurtdışı')
        train_df[col] = train_df[col].replace('İstanbul\nNiğde', 'Niğde')
        train_df[col] = train_df[col].replace('Çirişhanee', 'Bursa')
        train_df[col] = train_df[col].replace('Bilkent', 'Ankara')
        train_df[col] = train_df[col].replace('Hacettepe', 'Ankara')
        train_df[col] = train_df[col].replace('Yenibosna', 'İstanbul')
        train_df[col] = train_df[col].replace('İsparta', 'Isparta')
        train_df[col] = train_df[col].replace('Stanbul', 'İstanbul')
        train_df[col] = train_df[col].replace('Diger', 'Diğer')
        train_df[col] = train_df[col].replace('İsparta', 'Isparta')
        train_df[col] = train_df[col].replace('İsparta', 'Isparta')
        train_df[col] = train_df[col].replace('İsparta', 'Isparta')
        train_df[col] = train_df[col].replace('İsparta', 'Isparta')
        train_df[col] = train_df[col].replace('İsparta', 'Isparta')
        train_df[col] = train_df[col].replace('İsparta', 'Isparta')
        train_df[col] = train_df[col].replace('İsparta', 'Isparta')
        train_df[col] = train_df[col].replace('İsparta', 'Isparta')
        train_df[col] = train_df[col].replace('İsparta', 'Isparta')
        train_df[col] = train_df[col].replace('Mersın', 'Mersin')
        train_df[col] = train_df[col].replace('Bilecikbozüyük', 'Bilecik')
        train_df[col] = train_df[col].replace('Iznik', 'Bursa')
        train_df[col] = train_df[col].replace('Izmır', 'İzmir')
        train_df[col] = train_df[col].replace('Istanbu', 'İstanbul')
        train_df[col] = train_df[col].replace('Montezuma', 'Diğer')
        train_df[col] = train_df[col].replace('Beypazari', 'Ankara')
        train_df[col] = train_df[col].replace('Istabul', 'İstanbul')
        train_df[col] = train_df[col].replace('Ord', 'Ordu')
        train_df[col] = train_df[col].replace('Erenköy', 'İstanbul')
        train_df[col] = train_df[col].replace('Beylikduzu', 'İstanbul')
        train_df[col] = train_df[col].replace('Peshawar', 'Pakistan')
        train_df[col] = train_df[col].replace('Hatay3', 'Hatay')
        train_df[col] = train_df[col].replace('Gümüşyaka', 'İstanbul')
        train_df[col] = train_df[col].replace('Bakıoğlu', 'Bartın')
        train_df[col] = train_df[col].replace('Sarigöl', 'Manisa')
        train_df[col] = train_df[col].replace('Seydi̇Kemer', 'Muğla')
        train_df[col] = train_df[col].replace('Isranbul', 'İstanbul')
        train_df[col] = train_df[col].replace('Kahmanmaraş', 'Kahramanmaraş')
        train_df[col] = train_df[col].replace('Kahramanmaas', 'Kahramanmaraş')
        train_df[col] = train_df[col].replace('Diyarakır', 'Diyarbakır')
        train_df[col] = train_df[col].replace('Kasyeri', 'Kayseri')
        train_df[col] = train_df[col].replace('Kinshasa', 'Kongo')
        train_df[col] = train_df[col].replace('Kocaei̇', 'Kocaeli̇')
        train_df[col] = train_df[col].replace('Cerkezkoy', 'Tekirdağ')
        train_df[col] = train_df[col].replace('Kastamony', 'Kastamonu')
        train_df[col] = train_df[col].replace('Kolej', 'Diğer')
        train_df[col] = train_df[col].replace('Cankiri', 'Çankırı')
        train_df[col] = train_df[col].replace('Kurucaşi̇Le', 'Bartın')
        train_df[col] = train_df[col].replace('Kuşadasi', 'Aydın')
        train_df[col] = train_df[col].replace('Kürkcü', 'Erzurum')
        train_df[col] = train_df[col].replace('Canakkale', 'Çanakkale')
        train_df[col] = train_df[col].replace('Denizl', 'Denizli')
        train_df[col] = train_df[col].replace(':Ankara', 'Ankara')
        train_df[col] = train_df[col].replace('Karabbük', 'Karabük')
        train_df[col] = train_df[col].replace('Luleburgaz', 'İstanbul')
        train_df[col] = train_df[col].replace('Malatua', 'Malatya')
        train_df[col] = train_df[col].replace('Bi̇Ngöl', 'Bingöl')
        train_df[col] = train_df[col].replace('Kahramanmraş', 'Kahramanmaraş')
        train_df[col] = train_df[col].replace('Shirvan', 'Azerbaycan')
        train_df[col] = train_df[col].replace('Kdz.Eregli', 'Zonguldak')
        train_df[col] = train_df[col].replace('Shkoder', 'Arnavutluk')
        train_df[col] = train_df[col].replace('Corly', 'Tekirdağ')
        train_df[col] = train_df[col].replace('Ferizaj', 'Kosova')
        train_df[col] = train_df[col].replace('Villefranche', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Viranşehie', 'Şanlıurfa')
        train_df[col] = train_df[col].replace('Duzce', 'Düzce')
        train_df[col] = train_df[col].replace('Antaya', 'Antalya')
        train_df[col] = train_df[col].replace('Yeşi̇Lköy', 'İstanbul')
        train_df[col] = train_df[col].replace('Ankata', 'Ankara')
        train_df[col] = train_df[col].replace('Fethıye', 'Muğla')
        train_df[col] = train_df[col].replace('Abd', 'Yurtdışı')
        train_df[col] = train_df[col].replace('İsyanbul', 'İstanbul')
        train_df[col] = train_df[col].replace('Zi̇Le', 'Tokat')
        train_df[col] = train_df[col].replace('Ankar', 'Ankara')
        train_df[col] = train_df[col].replace('Almanya', 'Yurtdışı')
        train_df[col] = train_df[col].replace('Aliaga', 'İzmir')
        train_df[col] = train_df[col].replace('İstanbulküçükçekmece', 'İstanbul')
        train_df[col] = train_df[col].replace('Alapli', 'Zonguldak')
        train_df[col] = train_df[col].replace('Yılmaz', 'Manisa')
        train_df[col] = train_df[col].replace('İst', 'İstanbul')
        train_df[col] = train_df[col].replace('Urun', 'Uşak')
        train_df[col] = train_df[col].replace('Atyrau', 'Kazakistan')
        train_df[col] = train_df[col].replace('Şanlıyrfa', 'Şanlıurfa')
        train_df[col] = train_df[col].replace('Germncik', 'Aydın')
        train_df[col] = train_df[col].replace('Si̇Nopboyabat', 'Si̇nop')
        train_df[col] = train_df[col].replace('Gebe', 'Gaziantep')
        train_df[col] = train_df[col].replace('Sumgayıt', 'Azerbaycan')
        train_df[col] = train_df[col].replace('Ateş', 'Kocaeli')
        train_df[col] = train_df[col].replace('Unye', 'Ordu')
        train_df[col] = train_df[col].replace('Tekirdağmuratlı', 'Tekirdağ')
        train_df[col] = train_df[col].replace('Ti̇Re', 'İzmir')
        train_df[col] = train_df[col].replace('Aantalya', 'Antalya')
        train_df[col] = train_df[col].replace('Tranzon', 'Trabzon')
        train_df[col] = train_df[col].replace('Dıyarbakır', 'Diyarbakır')
        train_df[col] = train_df[col].replace('Esme', 'Uşak')
        train_df[col] = train_df[col].replace('Tursunzade', 'Tacikistan')
        train_df[col] = train_df[col].replace('Gazi̇Paşa', 'Antalya')
        train_df[col] = train_df[col].replace('Adi̇Yaman', 'Adıyaman')
        train_df[col] = train_df[col].replace('İstabul', 'İstanbul')
        train_df[col] = train_df[col].replace('Zakatala', 'Azerbaycan')
        train_df[col] = train_df[col].replace('Bi̇Tli̇S', 'Bitlis')
        train_df[col] = train_df[col].replace('Artvi̇N', 'Artvi̇n')
        train_df[col] = train_df[col].replace('Si̇İRt', 'Si̇irt')
        train_df[col] = train_df[col].replace('İstanbuk', 'İstanbul')
        train_df[col] = train_df[col].replace('Gi̇Resun', 'Gi̇resun')
        train_df[col] = train_df[col].replace('Erzi̇Ncan', 'Erzi̇ncan')
        train_df[col] = train_df[col].replace('Ni̇Ğde', 'Ni̇ğde')
        train_df[col] = train_df[col].replace('Nevşehi̇R', 'Nevşehi̇r')
        train_df[col] = train_df[col].replace('Si̇Nop', 'Si̇nop')
        train_df[col] = train_df[col].replace('Bi̇Leci̇K', 'Bi̇leci̇k')
        train_df[col] = train_df[col].replace('Si̇Vas', 'Sivas')
        train_df[col] = train_df[col].replace('Ri̇Ze', 'Ri̇ze')
        train_df[col] = train_df[col].replace('Osmani̇Ye', 'Osmani̇ye')
        train_df[col] = train_df[col].replace('Teki̇Rdağ', 'Teki̇rdağ')
        train_df[col] = train_df[col].replace('Deni̇Zli̇', 'Deni̇zli̇')
        train_df[col] = train_df[col].replace('Eski̇Şehi̇R', 'Eskişehir')
        train_df[col] = train_df[col].replace('Di̇Yarbakir', 'Diyarbakır')
        train_df[col] = train_df[col].replace('Mani̇Sa', 'Mani̇sa')
        train_df[col] = train_df[col].replace('Mersi̇N', 'Mersi̇n')
        train_df[col] = train_df[col].replace('Kırşehi̇R', 'Kırşehi̇r')
        train_df[col] = train_df[col].replace('İzmi̇R', 'İzmi̇r')


    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('İLKOKUL MEZUNU', 'İlkokul')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('İlkokul Mezunu', 'İlkokul')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('Eğitimi yok', 'İlkokul')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('Eğitim Yok', 'İlkokul')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('EĞİTİM YOK', 'İlkokul')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('0', 'İlkokul')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('Ortaokul Mezunu', 'Ortaokul')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('ORTAOKUL MEZUNU', 'Ortaokul')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('LİSE', 'Lise')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('Lise Mezunu', 'Lise')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('Üniversite Mezunu', 'Üniversite')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('ÜNİVERSİTE', 'Üniversite')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('YÜKSEK LİSANS', 'Yüksek Lisans')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('DOKTORA', 'Doktora')
    train_df["Baba Egitim Durumu"] = train_df["Baba Egitim Durumu"].replace('Yüksek Lisans / Doktara', 'Yüksek Lisans / Doktora')



    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('İLKOKUL MEZUNU', 'İlkokul')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('İlkokul Mezunu', 'İlkokul')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('Eğitimi yok', 'İlkokul')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('Eğitim Yok', 'İlkokul')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('EĞİTİM YOK', 'İlkokul')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('0', 'İlkokul')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('Ortaokul Mezunu', 'Ortaokul')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('ORTAOKUL MEZUNU', 'Ortaokul')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('LİSE', 'Lise')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('Lise Mezunu', 'Lise')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('Üniversite Mezunu', 'Üniversite')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('ÜNİVERSİTE', 'Üniversite')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('YÜKSEK LİSANS', 'Yüksek Lisans')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('DOKTORA', 'Doktora')
    train_df["Anne Egitim Durumu"] = train_df["Anne Egitim Durumu"].replace('Yüksek Lisans / Doktara', 'Yüksek Lisans / Doktora')


    train_df["Baba Sektor"] = train_df["Baba Sektor"].replace('ÖZEL SEKTÖR', 'Özel')
    train_df["Baba Sektor"] = train_df["Baba Sektor"].replace('Özel Sektör', 'Özel')
    train_df["Baba Sektor"] = train_df["Baba Sektor"].replace('KAMU', 'Kamu')
    train_df["Baba Sektor"] = train_df["Baba Sektor"].replace('ÖZEL SEKTÖR', 'Özel')
    train_df["Baba Sektor"] = train_df["Baba Sektor"].replace('DİĞER', 'Diğer')

    train_df['Spor Dalindaki Rolunuz Nedir?'] = train_df['Spor Dalindaki Rolunuz Nedir?'].replace('Lider/Kaptan', 'Lider')
    train_df['Spor Dalindaki Rolunuz Nedir?'] = train_df['Spor Dalindaki Rolunuz Nedir?'].replace('Kaptan', 'Lider')
    train_df['Spor Dalindaki Rolunuz Nedir?'] = train_df['Spor Dalindaki Rolunuz Nedir?'].replace('KAPTAN / LİDER', 'Lider')
    train_df['Spor Dalindaki Rolunuz Nedir?'] = train_df['Spor Dalindaki Rolunuz Nedir?'].replace('DİĞER', 'Diğer')
    train_df['Spor Dalindaki Rolunuz Nedir?'] = train_df['Spor Dalindaki Rolunuz Nedir?'].fillna("YOK")
    train_df['Spor Dalindaki Rolunuz Nedir?'] = train_df['Spor Dalindaki Rolunuz Nedir?'].replace('0', 'YOK')


    train_df['Kardes Sayisi'] = train_df['Kardes Sayisi'].fillna(0)
    train_df['Kardes Sayisi'] = train_df['Kardes Sayisi'].replace('Kardeş Sayısı 1 Ek Bilgi Aile Hk. Anne Vefat', 1)
    train_df['Kardes Sayisi'] = train_df['Kardes Sayisi'].astype(int)

    train_df["Anne Sektor"] = train_df["Anne Sektor"].replace('ÖZEL SEKTÖR', 'Özel')
    train_df["Anne Sektor"] = train_df["Anne Sektor"].replace('Özel Sektör', 'Özel')
    train_df["Anne Sektor"] = train_df["Anne Sektor"].replace('KAMU', 'Kamu')
    train_df["Anne Sektor"] = train_df["Anne Sektor"].replace('ÖZEL SEKTÖR', 'Özel')
    train_df["Anne Sektor"] = train_df["Anne Sektor"].replace('DİĞER', 'Diğer')

    train_df['Burs Aldigi Baska Kurum'] = train_df['Burs Aldigi Baska Kurum'].apply(
    lambda x: 'KYK' if pd.notna(x) and 'kyk' in x.lower() else x
    )

    train_df['Burs Aldigi Baska Kurum'] = train_df['Burs Aldigi Baska Kurum'].apply(group_similar_burs)

    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('Kredi ve Yurtlar Kurumu', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('Kredi Yurtlar Kurumu', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('Kredi yurtlar kurumu', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('Kredi ve yurtlar kurumu', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('K', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('kredi yurtlar kurumu', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('kredi ve yurtlar kurumu', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('Kredi ve yurtlar kurumu', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('KREDİ VE YURTLAR KURUMU', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('Devlet bursu', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('devlet', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('KREDİ YURTLAR KURUMU', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('Kredi ve Yurtlar Genel Müdürlüğü', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('Devlet Bursu', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('DEVLET', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('devlet bursu', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('Kredi', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('KYL', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('KYJ', 'KYK')
    train_df["Burs Aldigi Baska Kurum"] = train_df["Burs Aldigi Baska Kurum"].replace('KYM', 'KYK')


    train_df['Lise Sehir'] = train_df['Lise Sehir'].str.strip().str.title()
    train_df['Lise Sehir'] = train_df['Lise Sehir'].str.split('/').str[0].str.strip()
    train_df['Lise Sehir'] = train_df['Lise Sehir'].str.split(' ').str[0].str.strip()
    train_df['Lise Sehir'] = train_df['Lise Sehir'].str.split('-').str[0].str.strip()
    train_df['Lise Sehir'] = train_df['Lise Sehir'].str.split('_').str[0].str.strip()
    train_df['Lise Sehir'] = train_df['Lise Sehir'].str.split(',').str[0].str.strip()

    # Perform specific replacements for cities in 'Lise Sehir'
    city_replacements = {
        'İstanbulbakırköy': 'İstanbul', 'Istanbul': 'İstanbul', 'Aliağaizmir': 'İzmir',
        'İzmiR': 'İzmir', 'İzmirkonak': 'İzmir', 'Yenimahalleankara': 'Ankara',
        'Bursayıldırım': 'Bursa', 'Yumurtalıkadana': 'Adana', 'Karaisalıadana': 'Adana',
        'Adanaseyhan': 'Adana', 'Konyaselçuklu': 'Konya', 'Diyarbakir': 'Diyarbakır',
        'Diyarbakırdicle': 'Diyarbakır', 'Si̇Lvan ': 'Silvan', 'Sanliurfa': 'Şanlıurfa',
        'Şanliurfa': 'Şanlıurfa', 'Ş.Urfa': 'Şanlıurfa', 'Malatyamerkez': 'Malatya',
        'Kahramanmaras': 'Kahramanmaraş', 'K.Maras': 'Kahramanmaraş', 'K.Maraş': 'Kahramanmaraş',
        'Agri': 'Ağrı', 'Ağri': 'Kahramanmaraş', 'Balikesir': 'Balıkesir', 'BalikesiR': 'Balıkesir',
        'Orduünye': 'Ordu', 'Eskisehir': 'Eskişehir', 'Adiyaman': 'Adıyaman', 'Aydin': 'Aydın',
        'Bartin': 'Bartın', 'Elazig': 'Elazığ', 'Tekirdag': 'Tekirdağ', 'Sirnak': 'Şırnak',
        'Şirnak': 'Şırnak', 'Nigde': 'Niğde', 'Kirikkale': 'Kırıkkale', 'Kirklareli': 'Kırklareli',
        'Igdir': 'Iğdır'
    }

    train_df['Lise Sehir'] = train_df['Lise Sehir'].replace(city_replacements)

    train_df['Universite Kacinci Sinif'] = train_df['Universite Kacinci Sinif'].replace('hazırlık', 'Hazırlık')
    train_df["Lise Turu"] = train_df["Lise Turu"].replace('Anadolu Lisesi', 'Anadolu')
    train_df["Lise Turu"] = train_df["Lise Turu"].replace('Anadolu lisesi', 'Anadolu')
    train_df["Lise Turu"] = train_df["Lise Turu"].replace('Düz lise', 'Düz')
    train_df["Lise Turu"] = train_df["Lise Turu"].replace('Düz Lise', 'Düz')

    train_df["Lise Turu"] = train_df["Lise Turu"].replace('Özel Lisesi', 'Özel')
    train_df["Lise Turu"] = train_df["Lise Turu"].replace('Özel lisesi', 'Özel')
    train_df["Lise Turu"] = train_df["Lise Turu"].replace('Özel Lise', 'Özel')

    train_df["Lise Turu"] = train_df["Lise Turu"].replace('Meslek lisesi', 'Meslek')
    train_df["Lise Turu"] = train_df["Lise Turu"].replace('Meslek Lisesi', 'Meslek')
    train_df["Lise Turu"] = train_df["Lise Turu"].replace('Fen lisesi', 'Meslek')
    train_df["Lise Turu"] = train_df["Lise Turu"].replace('Fen Lisesi', 'Meslek')

    train_df['Cinsiyet'] = train_df['Cinsiyet'].replace('ERKEK', 'Erkek')

    train_df['Burs Aliyor mu?'] = train_df['Burs Aliyor mu?'].replace('EVET', 'Evet')
    train_df['Burs Aliyor mu?'] = train_df['Burs Aliyor mu?'].replace('evet', 'Evet')
    train_df['Burs Aliyor mu?'] = train_df['Burs Aliyor mu?'].replace('hayır', 'Hayır')

    train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].replace('ORTALAMA BULUNMUYOR', 'YOK')
    train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].replace('Not ortalaması yok', 'YOK')
    train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].replace('Ortalama bulunmuyor', 'YOK')
    train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].replace('3.00-2.50', '2.50 - 3.00')
    train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].replace('3.50-3', '3.00 - 3.50')
    train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].replace('3.00 - 3.49', '3.00 - 3.50')
    train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].replace('2.50 -3.00', '2.50 - 3.00')
    train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].replace('2.50 - 2.99', '2.50 - 3.00')
    train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].replace('4-3.5', '3.50 - 4.00')
    train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].replace('4.0-3.5', '3.50 - 4.00')

    train_df['Universite Turu'] = train_df['Universite Turu'].replace('Devlet', 'DEVLET')
    train_df['Universite Turu'] = train_df['Universite Turu'].replace('Özel', 'ÖZEL')

    train_df['Lise Bolumu'] = train_df['Lise Bolumu'].apply(group_similar_lise)


    train_df['Bölüm'] = train_df['Bölüm'].apply(safe_standardize_name)
    train_df['Universite Adi'] = train_df['Universite Adi'].apply(safe_standardize_university_name)
    train_df['Bölüm'] = train_df['Bölüm'].apply(group_similar_departments)


    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARBILIMIVEMUHENDISLIGI', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHENDISLIG', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUH', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHENDISLI', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARVEYAZILIMMUHENDISLIGI', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHENDISI', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHENDISL', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARM', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHENDI', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('YONETIMBILISIMSISTEMLERIBILGISAYARMUHILEDEYANDALYAPIYORUM', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('LISANSBILGISAYARMUHYLISANSELEKTRIKELEKTRONIKMUH', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARSISTEMLERIVEMUHENDI', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHENG', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHENDISLIG', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHNEDISLIGI', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHINGILIZCE', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHING', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHENSILIGI', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARELEKTRIKELEKTRONIKMUHENDISLIGI', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHE', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHENDIS', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHENDILS', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHENDILIGI', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMUHEN', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMU', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARMGHEN', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYARVEBILISIMMUHENDISLIGI', 'BILGISAYAR MUHENDISLIĞI')
    train_df['Bölüm'] = train_df['Bölüm'].replace('BILGISAYAR MUHENDISLIĞI', 'BILGISAYAR MUHENDISLIĞI')



    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKINGILIZCE', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKBURSLU', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKINGILIZCETAMBURSLU')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKBOLUMU', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKTAMBURSLU', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKINGILIZCEBURSLU', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKCIFTANADALPROGRAMIOGRENCISIYIMMIMARLIKEGITIMIMITAMAMLADIMINSAATMUHENDISLIGIEGITIMIMISURDURUYORUM', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKCIFTANADALPROGRAMI', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKFAKU', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKFAKULTESI', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKFAKULTESIMIMARLIK', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKFAKULTESIYUKSEKLISANS', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKHAYALLERIMINMESLEGIDESEKDAHADOGRUOLUR', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKING', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKOKUYORUMSADECE', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKVEICMIMARLIKCIFTANADAL', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKYUKSEKLISAN', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MUHENDISLIKVEMIMARLIKFAKULTESIMIMARLIK', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('INGILIZCEMIMARLIK', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLI', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARL', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKB', 'MIMARLIK')
    train_df['Bölüm'] = train_df['Bölüm'].replace('MIMARLIKCIFTANADALPROGRAMI', 'MIMARLIK')


    # train_df['ANNE_BABA_EGITIM'] = train_df['Anne Egitim Durumu'].astype(str) + '_' + train_df['Baba Egitim Durumu'].astype(str)


    train_df = lda_score(train_df)
    train_df = girisimcilik_process(train_df)

    # train_df = stk_process(train_df)

    # train_df = lda_score_2(train_df)
    # train_df = ngram_process(train_df)
    # train_df = ner_process(train_df)
    # train_df = apply_tfidf(train_df)

    return train_df

In [12]:
train_df[Config.target_col].fillna(0, inplace=True)

train_df = process_data(train_df)
test_df = process_data(test_df)

train_df = train_df.drop(columns=Config.drop_cols)
test_df = test_df.drop(columns=Config.drop_cols)


cat_cols_train= train_df.select_dtypes(include=['object']).columns
cat_cols_test = test_df.select_dtypes(include=['object']).columns

for col in cat_cols_train:
    train_df[col] = train_df[col].astype(str).fillna('Missing')
    train_df[col] = train_df[col].astype('category')


for col in cat_cols_test:
    test_df[col] = test_df[col].astype(str).fillna('Missing')
    test_df[col] = test_df[col].astype('category')


In [13]:
len(train_df['Lise Bolumu'].value_counts())
# 360

11

In [14]:
train_df['Lise Bolumu'].value_counts()
# 34459

Lise Bolumu
sayısal                   35090
eşit ağırlık              20944
sözel                      4228
None                       2853
dil bölümü                 1517
endüstri meslek lisesi      454
çocuk gelişimi               26
öğretmen                      5
imam hatip lisesi             4
askeri                        2
sosyal bilimler               2
Name: count, dtype: int64

In [15]:
def full_fit_cb(data, target_col):
    X = data.drop(target_col, axis=1)
    y = data[target_col]

    cat_features = [X.columns.get_loc(col) for col in X.select_dtypes(include=['category'])]

    print("Training model on full dataset...")
    model = cb.CatBoostRegressor(**Config.catboost_params)

    model.fit(X, y,
                cat_features=cat_features,
                verbose=100)

    print("Training completed.")

    # If you want to calculate training RMSE
    train_preds = model.predict(X)
    train_rmse = sqrt(mean_squared_error(y, train_preds))
    print(f'Training RMSE: {train_rmse:.4f}')

    return model

In [16]:
def train_catboost(data, target_col):
    
    X = data.drop(target_col, axis=1)
    y = data[target_col]
        
    cv = KFold(n_splits=5, random_state=42, shuffle=True)
    models = []
    oof_preds = np.zeros(X.shape[0])
    metrics = []
    
    cat_features = [X.columns.get_loc(col) for col in X.select_dtypes(include=['category'])]
    
    for fi, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        
        print(f'Fold {fi+1}/{5} ...')
        
        model = cb.CatBoostRegressor(**Config.catboost_params)
        
        model.fit(X.iloc[train_idx], y.iloc[train_idx],
                  eval_set=(X.iloc[valid_idx], y.iloc[valid_idx]),
                  use_best_model=True,
#                   early_stopping_rounds=Config.early_stop,
                  cat_features = cat_features,
                  verbose=100)
        
        preds = model.predict(X.iloc[valid_idx])
        
        oof_preds[valid_idx] = preds
        
        
        rmse = sqrt(mean_squared_error(y.iloc[valid_idx], preds))
        metrics.append(rmse)
        print(f'Fold {fi+1} RMSE: {rmse:.4f}')
        
        models.append(model)
        
        
    oof_rmse = sqrt(mean_squared_error(y, oof_preds))
    
    print(f'Average RMSE: {np.mean(metrics):.4f}')
    print(f'OOF RMSE: {oof_rmse:.4f}')
    
    return models

In [17]:
def infer_models(data, catboost_models):
    
#     xgb_data = data.copy()
#     dxgb = xgb.DMatrix(xgb_data, enable_categorical=True)
    # Ensure the predictions are not empty
    catboost_preds = np.mean([model.predict(data) for model in catboost_models], axis=0)
#     lightgbm_preds = np.mean([model.predict(data) for model in lightgbm_models], axis=0)
#     xgboost_preds = np.mean([model.predict(dxgb) for model in xgboost_models], axis=0)
    # combined_preds = (catboost_preds + lightgbm_preds + xgboost_preds) / 3
    combined_preds = catboost_preds
    
    # Debug: Check length of predictions
    print(f'Predictions length: {len(combined_preds)}')  # Ensure this matches the length of data
    
    return combined_preds

In [18]:
def full_fit_infer(data, target_col, catboost_model):
  # Ensure we're using the correct columns
    X = data.drop(target_col, axis=1) if target_col in data.columns else data

      # Make predictions using the full model
    predictions = catboost_model.predict(X)

      # Debug: Check length of predictions
    print(f'Predictions length: {len(predictions)}')  # Ensure this matches the length of data

    return predictions


In [19]:
import os

# Get the number of CPU cores
num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")


Number of CPU cores: 4


In [20]:
# model = train_catboost(train_df, Config.target_col)
# predictions = infer_models(test_df, Config.target_col, model)

In [21]:
time_limit = 3600*16

predictor = TabularPredictor(label=Config.target_col, problem_type='regression', eval_metric='root_mean_squared_error')
predictor.fit(train_df, presets='medium_quality', time_limit=time_limit, num_bag_folds=5, num_bag_sets=1, num_stack_levels=1, dynamic_stacking=False,
           included_model_types=['CAT', 'XGB', 'GBM', 'RF', 'XT'], ag_args_fit={'num_gpus': 1, 'num_cpus': 4},
           hyperparameters = {
              'GBM': [
                    {'max_depth': 6, 'device': 'cpu'},
                    {'max_depth': 5, 'device': 'cpu'},
                    {'max_depth': 4, 'device': 'cpu'},
                    {'max_depth': 3, 'device': 'cpu'},
              ],
              'XGB': [
                  {'max_depth': 6},
                  {'max_depth': 5},
                  {'max_depth': 4},
                  {'max_depth': 3},
#                   {'max_depth': 2},
              ],
              'CAT': [
                  {'depth': 6},
                  {'depth': 5},
                  {'depth': 4},
                  {'depth': 3},
#                   {'depth': 2},
              ],
              'RF': [
                  {'max_depth': 6},
                  {'max_depth': 5},
                  {'max_depth': 4},
                  {'max_depth': 3},
#                   {'depth': 2},
              ],
              'XT': [
                  {'max_depth': 6},
                  {'max_depth': 5},
                  {'max_depth': 4},
                  {'max_depth': 3},
#                   {'depth': 2},
              ],
           }
          )

No path specified. Models will be saved in: "AutogluonModels/ag-20240915_181740"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       29.74 GB / 31.36 GB (94.9%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 57600s
AutoGluon will save models to "AutogluonModels/ag-20240915_181740"
Train Data Rows:    65125
Train Data Columns: 44
Label Column:       Degerlendirme Puani
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    30481.98 MB
	Train Data (Original)  Memory Usage: 25.22 MB (0.1% of available memory)
	Inferring data type of each featur

[1000]	valid_set's rmse: 6.55195
[1000]	valid_set's rmse: 6.49061
[1000]	valid_set's rmse: 6.6078


	-6.5384	 = Validation score   (-root_mean_squared_error)
	31.22s	 = Training   runtime
	2.82s	 = Validation runtime
Fitting model: LightGBM_4_BAG_L1 ... Training model for up to 38309.79s of the 57519.25s of remaining time.
	Fitting 5 child models (S1F1 - S1F5) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's rmse: 6.6109
[2000]	valid_set's rmse: 6.54221
[1000]	valid_set's rmse: 6.63034
[2000]	valid_set's rmse: 6.55958
[1000]	valid_set's rmse: 6.60948
[2000]	valid_set's rmse: 6.51224
[1000]	valid_set's rmse: 6.6031
[2000]	valid_set's rmse: 6.53104
[1000]	valid_set's rmse: 6.64823
[2000]	valid_set's rmse: 6.57988


	-6.5417	 = Validation score   (-root_mean_squared_error)
	50.13s	 = Training   runtime
	5.06s	 = Validation runtime
Fitting model: RandomForest_BAG_L1 ... Training model for up to 38253.9s of the 57463.37s of remaining time.
	-8.0542	 = Validation score   (-root_mean_squared_error)
	21.94s	 = Training   runtime
	1.24s	 = Validation runtime
Fitting model: RandomForest_2_BAG_L1 ... Training model for up to 38230.59s of the 57440.05s of remaining time.
	-8.8285	 = Validation score   (-root_mean_squared_error)
	19.08s	 = Training   runtime
	1.2s	 = Validation runtime
Fitting model: RandomForest_3_BAG_L1 ... Training model for up to 38210.18s of the 57419.65s of remaining time.
	-9.5848	 = Validation score   (-root_mean_squared_error)
	14.99s	 = Training   runtime
	1.16s	 = Validation runtime
Fitting model: RandomForest_4_BAG_L1 ... Training model for up to 38193.9s of the 57403.36s of remaining time.
	-10.7153	 = Validation score   (-root_mean_squared_error)
	12.02s	 = Training   runtime


In [22]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-6.228961,root_mean_squared_error,34.785135,2146.895152,0.001203,0.201734,3,True,42
1,XGBoost_2_BAG_L2,-6.241165,root_mean_squared_error,32.001021,1822.158880,0.854028,9.333592,2,True,39
2,XGBoost_BAG_L2,-6.242535,root_mean_squared_error,32.247615,1822.246212,1.100622,9.420924,2,True,38
3,CatBoost_BAG_L2,-6.242591,root_mean_squared_error,32.015239,1965.532296,0.868246,152.707009,2,True,30
4,CatBoost_2_BAG_L2,-6.243133,root_mean_squared_error,31.961035,1975.231893,0.814043,162.406605,2,True,31
5,XGBoost_3_BAG_L2,-6.246603,root_mean_squared_error,32.023358,1822.563193,0.876366,9.737905,2,True,40
6,CatBoost_3_BAG_L2,-6.248764,root_mean_squared_error,31.853551,1943.518439,0.706558,130.693151,2,True,32
7,XGBoost_4_BAG_L2,-6.251863,root_mean_squared_error,32.050136,1824.215117,0.903143,11.389829,2,True,41
8,CatBoost_4_BAG_L2,-6.257130,root_mean_squared_error,31.727331,1911.994879,0.580339,99.169592,2,True,33
9,ExtraTrees_BAG_L2,-6.265675,root_mean_squared_error,32.829750,1823.993661,1.682757,11.168373,2,True,34


In [23]:
predictor.refit_full(model="best")

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: CatBoost_BAG_L1_FULL ...
	644.46s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: XGBoost_BAG_L1_FULL ...
	4.55s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBM_BAG_L1_FULL ...
	3.85s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBM_2_BAG_L1_FULL ...
	4.46s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBM_3_BAG_L1_FULL ...
	7.0s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBM_4_BAG_L1_FULL ...
	11.89s	 = Training   runtime
Fitting model: Ra

{'CatBoost_BAG_L1': 'CatBoost_BAG_L1_FULL',
 'XGBoost_BAG_L1': 'XGBoost_BAG_L1_FULL',
 'LightGBM_BAG_L1': 'LightGBM_BAG_L1_FULL',
 'LightGBM_2_BAG_L1': 'LightGBM_2_BAG_L1_FULL',
 'LightGBM_3_BAG_L1': 'LightGBM_3_BAG_L1_FULL',
 'LightGBM_4_BAG_L1': 'LightGBM_4_BAG_L1_FULL',
 'RandomForest_BAG_L1': 'RandomForest_BAG_L1_FULL',
 'RandomForest_2_BAG_L1': 'RandomForest_2_BAG_L1_FULL',
 'RandomForest_3_BAG_L1': 'RandomForest_3_BAG_L1_FULL',
 'RandomForest_4_BAG_L1': 'RandomForest_4_BAG_L1_FULL',
 'CatBoost_2_BAG_L1': 'CatBoost_2_BAG_L1_FULL',
 'CatBoost_3_BAG_L1': 'CatBoost_3_BAG_L1_FULL',
 'CatBoost_4_BAG_L1': 'CatBoost_4_BAG_L1_FULL',
 'ExtraTrees_BAG_L1': 'ExtraTrees_BAG_L1_FULL',
 'ExtraTrees_2_BAG_L1': 'ExtraTrees_2_BAG_L1_FULL',
 'ExtraTrees_3_BAG_L1': 'ExtraTrees_3_BAG_L1_FULL',
 'ExtraTrees_4_BAG_L1': 'ExtraTrees_4_BAG_L1_FULL',
 'XGBoost_2_BAG_L1': 'XGBoost_2_BAG_L1_FULL',
 'XGBoost_3_BAG_L1': 'XGBoost_3_BAG_L1_FULL',
 'XGBoost_4_BAG_L1': 'XGBoost_4_BAG_L1_FULL',
 'CatBoost_BAG_L2': 

In [24]:
predictions = predictor.predict(test_df)

In [25]:
sub_df = pd.DataFrame({
    'id': range(len(test_df)),
    'Degerlendirme Puani': predictions
})

In [26]:
sub_df.to_csv('submission.csv', index=False)

In [27]:
sub = pd.read_csv('/kaggle/working/submission.csv')
sub.head()

,id,Degerlendirme Puani
0,0,32.958546
1,1,20.865759
2,2,8.486250
3,3,18.391670
4,4,42.647633
